In this lecture we study how to scrape Yelp reviews:

In [1]:
import webbrowser
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import bs4
import json

pd.options.display.max_rows = 999

In [2]:
#res = requests.get('https://www.yelp.com/biz/froedtert-hospital-milwaukee') 
res = requests.get('https://www.yelp.com/biz/rush-university-medical-center-chicago-3?osq=Rush+University+Hospital')
try:
    res.raise_for_status()
except Exception as exc:
    print('There was a problem: %s' % (exc))
soup = bs4.BeautifulSoup(res.text, 'html.parser')

Here we need to press F12 on the keyboard and look at the html file. We will need to identify where the reviews all are. They are stored in some type of json files:

In [3]:
data = soup.find_all('script', type='application/ld+json')
data2=list(data[1].children)
data3=json.loads(data2[0]) # this can only take a list and change it to a dictionary
data3.keys()

dict_keys(['aggregateRating', 'review', 'priceRange', 'name', 'address', '@context', 'image', '@type', 'telephone'])

In [4]:
print(data3['review'][0], '\n') # this is a list
print(data3['review'][1], '\n') # this is a list
print(data3['review'][2], '\n') # this is a list

{'reviewRating': {'ratingValue': 5}, 'datePublished': '2018-08-21', 'description': "I came here for back surgery and kept overnight for observation. The medical staff was really nice. I'm in a room but only in an area for 1 night so the room isn't all that. Then again it is private and everyone kept me comfortable.\n\nNurses are great and even though I could tell a few were tired they were friendly and made sure I had whatever I needed.\n\nThe layout of the hospital got me lost initially but everyone helped me to where I need to go.\n\nI came in for surgery last week BUT I had a heart murmur and I had to cancel and come back the other day for surgery. No one has ever told me I had one before.\n\nThey did an echocardiogram on me last Thursday they were going to do it with contrast and the person who put in my IV had a blow out so I have a huge bruise. Not too bad. \n\nThey went ahead with surgery, satisfaction on the echocardiogram. Cleared for surgery. \n\nThe food is pretty decent. I'

In [5]:
data4=data3['review']
print(data4[0]['reviewRating']['ratingValue'])
print(data4[0]['datePublished'])
print(data4[0]['description'])

5
2018-08-21
I came here for back surgery and kept overnight for observation. The medical staff was really nice. I'm in a room but only in an area for 1 night so the room isn't all that. Then again it is private and everyone kept me comfortable.

Nurses are great and even though I could tell a few were tired they were friendly and made sure I had whatever I needed.

The layout of the hospital got me lost initially but everyone helped me to where I need to go.

I came in for surgery last week BUT I had a heart murmur and I had to cancel and come back the other day for surgery. No one has ever told me I had one before.

They did an echocardiogram on me last Thursday they were going to do it with contrast and the person who put in my IV had a blow out so I have a huge bruise. Not too bad. 

They went ahead with surgery, satisfaction on the echocardiogram. Cleared for surgery. 

The food is pretty decent. I've been to many places that were bad. The foods here are easy to order from your ph

In [6]:
for k in range(0,len(data4)):
    del(data4[k]['author']) # gettiing rid of authorship
    data4[k]['StarRating']=data4[k]['reviewRating']['ratingValue'] # making a new key from existing key
    del(data4[k]['reviewRating']) # getting rid of the existing key since the value of the key is a dictionary

In [7]:
pd.options.display.max_rows=999
df=pd.DataFrame(data4)
df.sort_values(by='StarRating', inplace=True)
df

,StarRating,datePublished,description
1,1,2019-01-15,After conforming several times that coverage w...
3,1,2019-01-04,The price is unreasonable. I just have a docto...
16,1,2017-10-18,"In May 2016, my parents visited my brother and..."
10,1,2018-07-09,Illegally balance billing Medicaid patients. I...
15,1,2018-02-01,Had a horrible time with Kidney Transplant Pro...
14,1,2018-09-13,Went to the dermatology department with a bad ...
5,2,2018-09-17,Just an update: my Yelp review got the attenti...
2,4,2018-11-05,Excellent health care and health focused train...
4,4,2018-06-20,I have been living with Multiple Sclerosis for...
7,4,2018-07-07,"The nurses and doctors here are great, .. the ..."


In [8]:
writer = pd.ExcelWriter('H:\\Hospital Yelp Review.xlsx')
df.to_excel(writer,'Yelp Review')
writer.save()